In [1]:
stream_path = 'data/train_mseed/GSOK029_8-2014.mseed'

In [2]:
from obspy import read
stream = read(stream_path)

In [4]:
sr =  stream[0].stats.sampling_rate

In [5]:
# STA / LTA params
freqmin = 1
freqmax = 15

In [6]:
st = stream.copy()
st.filter('bandpass', freqmin=freqmin, freqmax=freqmax, corners=2, zerophase=True)
st.detrend(type='simple')
st.detrend(type='demean')

32 Trace(s) in Stream:

GS.OK029.00.HH1 | 2014-08-01T00:00:00.005000Z - 2014-08-05T19:59:12.075000Z | 100.0 Hz, 41755208 samples
...
(30 other traces)
...
GS.OK029.00.HHZ | 2014-08-27T18:19:15.425000Z - 2014-08-31T23:59:58.995000Z | 100.0 Hz, 36604358 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]

In [39]:
minnumsta = 1
tron = 1.5 
troff = 1 
sta = 1     * sr     # So it's in seconds
lta = 10    * sr     # So it's in seconds

In [40]:
from obspy.signal.trigger import coincidence_trigger

triglist = coincidence_trigger("recstalta", tron, troff, st, minnumsta, sta=sta, lta=lta)

In [41]:
times = [trig['time'] for trig in triglist]
print(len(times), 'detections')

821 detections


## Analyze Detections

In [42]:
import sys
sys.path.append('../analyze-benz')
import catalog_lookup as cl

In [43]:
start = stream[0].stats.starttime
end =  stream[-1].stats.endtime

start = start.strftime('%m/%d/%Y')
end = end.strftime('%m/%d/%Y')

In [44]:
catalog_times = cl.catalog_df('data/Benz_catalog.csv', start, end)

In [45]:
range_before = 10   # Finding in catalog start x seconds before time
range_after = 10    # Finding in catalog can start x seconds after time

In [46]:
import pandas as pd

predicted_times = pd.DataFrame(times, columns=['event_start']) 
predicted_times['event_end'] = predicted_times['event_start'] + range_after
predicted_times['event_start'] = predicted_times['event_start'] - range_before
predicted_times.head()

,event_start,event_end
0,2014-08-01T00:16:30.005000Z,2014-08-01T00:16:50.005000Z
1,2014-08-01T02:33:03.735000Z,2014-08-01T02:33:23.735000Z
2,2014-08-01T05:15:07.345000Z,2014-08-01T05:15:27.345000Z
3,2014-08-01T10:39:36.585000Z,2014-08-01T10:39:56.585000Z
4,2014-08-01T14:19:47.245000Z,2014-08-01T14:20:07.245000Z


In [47]:
predicted_events = cl.find_events_in_catalog(predicted_times, catalog_times)

In [48]:
pe_found = predicted_events[predicted_events['catalog_id'] != -1]

In [49]:
from itertools import chain

found_indexes = set(chain.from_iterable(pe_found['catalog_id']))   # Flatten list of lists
catalog_events = catalog_times.copy()
catalog_events['found'] = catalog_events.index.map(lambda i: i in found_indexes)

In [50]:
found = catalog_events[catalog_events['found'] == True]
missed = catalog_events[catalog_events['found'] == False]
_new = predicted_events[predicted_events['catalog_id'] == -1]

In [51]:
len(found), len(missed), len(_new)

(555, 11566, 344)